In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import re

In [21]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

def extract_data(root):
    data = pd.DataFrame()
    test = []
    year = re.search(r'\d{4}', root.tag).group()
    for question in root.findall('question'):
        question_id = question.get('id')
        ce = question.get('CE')
        ds = question.get('DS')
        ek = question.get('EK')
        ic = question.get('IC')
        mr = question.get('MR')
        tc = question.get('TC')
        image = question.get('image')
        header = question.find('header').text
        statement = question.find('statement').text
        # Extrair as opções de resposta
        options = []
        correct_answer = None
        options = {}
        for option in question.findall('answers/option'):
            option_id = option.get('id')
            option_text = option.text
            option_correct = option.get('correct')
            options.update({option_id: option_text})
            #options.append({option_id: option_text})
            if option_correct == 'Yes':
                correct_answer = {option_id: option_text}
        # Armazenar os dados nas listas
        data = pd.concat([data, pd.DataFrame([{
            'id': question_id,
            'year': int(year),
            'context': header,
            'question': statement,
            'options': options,
            'correct_answer': correct_answer,
            'CE': ce,
            'DS': ds,
            'EK': ek,
            'IC': ic,
            'MR': mr,
            'TC': tc,
            'image': image
        }])], ignore_index=True)
    return data

def to_dataframe(data):
    df = pd.DataFrame(data)
    return df

def to_csv(df, filename):
    df.to_csv(filename, index=False)

In [23]:
directory = 'ENEMdataset'
cols = ['id',
        'year',
        'context',
        'question',
        'options',
        'correct_answer',
        'CE',
        'DS',
        'EK',
        'IC',
        'MR',
        'TC',
        'image']
df = pd.DataFrame(columns=cols)
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        filepath = os.path.join(directory, filename)
        df_2 = extract_data(parse_xml(filepath))
        df = pd.concat([df,df_2], join="outer", sort=False, ignore_index=True)

In [29]:
df.describe()

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image
count,1754,1754,1751,1754,1754,1754,1754,1754,1754,1754,1754,1754,1754
unique,180,9,1747,1754,1703,1698,2,2,2,2,2,2,2
top,96,2016,Disponível em: http://www.wordinfo.info. Acess...,\nOs mapas representam distintos padrões ...,"{'A': ' B- C- D- E- ', 'B': None, 'C': None, '...",{'E': None},No,No,No,No,No,Yes,No
freq,10,350,2,1,20,12,1711,1372,1251,1162,1244,880,1086


In [31]:
df.context[96]

'Disponível em: www.ccsp.com.br. Acesso em: 26 jul. 2010 (adaptado).\n'

In [40]:
df.options[1]

{'A': '  crescimento da população urbana e aumento da especulação imobiliária.\n',
 'B': '  direcionamento maior do fl uxo de pessoas, devido à existência de um grande número de serviços.\n',
 'C': '  delimitação de áreas para uma ocupação organizada do espaço físico, melhorando a qualidade de vida.\n',
 'D': '  implantação de políticas públicas que promovem a moradia e o direito à cidade aos seus moradores.\n',
 'E': '  reurbanização de moradias nas áreas centrais, mantendo o trabalhador próximo ao seu emprego, diminuindo os deslocamentos para a periferia.\n\n'}

In [34]:
to_csv(df, 'provas_enem.csv')

In [41]:
df

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image
0,01,2013,\nDistribuição espacial atual da população bra...,\nOs mapas representam distintos padrões ...,"{'A': ' fertilização natural dos solos. ', 'B...",{'E': ' concentração histórica da urbanização...,No,No,No,Yes,No,No,Yes
1,02,2013,\nTrata-se de um gigantesco movimento de const...,\nA dinâmica de transformação das cidades...,{'A': ' crescimento da população urbana e aum...,{'A': ' crescimento da população urbana e aum...,No,No,Yes,No,No,No,No
2,03,2013,"\nNo dia 1º de julho de 2012, a cidade do Rio ...",\nO reconhecimento da paisagem em questão...,"{'A': ' presença do corpo artístico local. ',...",{'E': ' relação sociedade-natureza de caráter...,No,No,Yes,No,No,Yes,No
3,04,2013,TEXTO I\n\nHá já algum tempo eu me aperc...,\nA exposição e a análise do projeto cartesian...,{'A': ' retomar o método da tradição para edi...,{'B': ' questionar de forma ampla e prof...,No,No,No,No,No,Yes,No
4,05,2013,"\nDe ponta a ponta, é tudo praia-palma, ...",\nA carta de Pero Vaz de Caminha permit...,{'A': ' Valorizar a catequese a ser real...,{'A': ' Valorizar a catequese a ser real...,No,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,177,2016,\nA London Eye é uma enorme roda-gigante na ca...,\nQual a medida que mais se aproxima do...,"{'A': ' 53 ', 'B': ' 94 ', 'C': ' 113 ', 'D': ...",{'D': ' 135 '},No,No,No,No,Yes,No,Yes
1750,176,2016,"\nEm uma empresa de móveis, um cliente encomen...","\nA altura, a largura e a profundidade do dese...","{'A': ' 22,00 cm, 12,00 cm e 5,00 cm. ', 'B': ...","{'A': ' 22,00 cm, 12,00 cm e 5,00 cm. '}",No,No,No,No,Yes,No,No
1751,178,2016,Os alunos de uma escola utilizaram cadeiras ig...,\nQual é o esboço obtido pelos alunos?\n,"{'A': ' B- C- D- E-', 'B': None, 'C': None, 'D...",{'C': None},No,No,No,Yes,No,No,Yes
1752,179,2016,\nPara garantir a segurança de um grande...,\nQuantos policiais serão necessários no iníci...,"{'A': ' 360 ', 'B': ' 485 ', 'C': ' 560 ', 'D'...",{'E': ' 860 '},No,No,No,No,Yes,No,No
